In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format,dayofweek


config = configparser.ConfigParser()
#config.read('dl.cfg')
config.read('/home/gari/.aws/credentials')
os.environ['AWS_ACCESS_KEY_ID']=config.get('credentials','KEY')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('credentials','SECRET')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('credentials','SECRET')

In [2]:

def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [3]:
spark = create_spark_session()
input_data = "s3a://udacity-dend/"

In [9]:
aa="""
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
"""

In [ ]:
new_aa=[each.split(" ")[-1]for each in aa.split(":")]
new_aa.pop(-1)

In [52]:
#exprs= [StructField("'{}'".format(field),StringType()) for field in new_aa]

In [29]:
exprs

[StructField('artist',StringType,true),
 StructField('auth',StringType,true),
 StructField('firstName',StringType,true),
 StructField('gender',StringType,true),
 StructField('itemInSession',StringType,true),
 StructField('lastName',StringType,true),
 StructField('length',StringType,true),
 StructField('level',StringType,true),
 StructField('location',StringType,true),
 StructField('method',StringType,true),
 StructField('page',StringType,true),
 StructField('registration',StringType,true),
 StructField('sessionId',StringType,true),
 StructField('song',StringType,true),
 StructField('status',StringType,true),
 StructField('ts',StringType,true),
 StructField('userAgent',StringType,true),
 StructField('userId',StringType,true)]

In [4]:
from pyspark.sql.types import StructType, StructField , DoubleType, StringType, IntegerType, DateType
#songSchema = StructType(exprs)

In [37]:
songSchema

StructType(List(StructField('artist',StringType,true),StructField('auth',StringType,true),StructField('firstName',StringType,true),StructField('gender',StringType,true),StructField('itemInSession',StringType,true),StructField('lastName',StringType,true),StructField('length',StringType,true),StructField('level',StringType,true),StructField('location',StringType,true),StructField('method',StringType,true),StructField('page',StringType,true),StructField('registration',StringType,true),StructField('sessionId',StringType,true),StructField('song',StringType,true),StructField('status',StringType,true),StructField('ts',StringType,true),StructField('userAgent',StringType,true),StructField('userId',StringType,true)))

In [8]:
df = spark.read.json("/home/gari/Desktop/myGItRepos/project_spark/data/song_data/*/*/*/*.json")

In [6]:
song_data_pandas=df.toPandas()

In [7]:
song_data_pandas.to_csv("./song_data.csv")

In [41]:
df.show(5)

+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|ARDR4AC1187FB371A1|           null|                    |            null|Montserrat Caball...|511.16363|        1|SOBAYLL12A8C138AF9|Sono andati? Fing...|   0|
|AREBBGV1187FB523D2|           null|         Houston, TX|            null|Mike Jones (Featu...|173.66159|        1|SOOLYAZ12A6701F4A6|Laws Patrolling (...|   0|
|ARMAC4T1187FB3FA4C|       40.82624|   Morris Plains, NJ|       -74.47995|The Dillinger Esc...|207.77751|        1|SOBBUGU12A8C13E95D|Setting Fire to S...|2004|
|ARPBNLO1187FB3D52F|       40.7145

In [ ]:
#import  pyspark.sql.functions as F
#df = df.withColumn("ts", F.to_timestamp("ts"))

In [65]:
df.createOrReplaceTempView("song_data")

In [66]:
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [73]:
spark.sql("""
SELECT song_id,
        title,
        artist_id,
        year,
        duration
        FROM song_data
""").limit(10).show(10)

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOBAYLL12A8C138AF9|Sono andati? Fing...|ARDR4AC1187FB371A1|   0|511.16363|
|SOOLYAZ12A6701F4A6|Laws Patrolling (...|AREBBGV1187FB523D2|   0|173.66159|
|SOBBUGU12A8C13E95D|Setting Fire to S...|ARMAC4T1187FB3FA4C|2004|207.77751|
|SOAOIBZ12AB01815BE|I Hold Your Hand ...|ARPBNLO1187FB3D52F|2000| 43.36281|
|SONWXQJ12A8C134D94|The Ballad Of Sle...|ARNF6401187FB57032|1994|  305.162|
|SONYPOM12A8C13B2D7|I Think My Wife I...|ARDNS031187B9924F0|2005|186.48771|
|SODREIN12A58A7F2E5|A Whiter Shade Of...|ARLTWXK1187FB5A3F8|   0|326.00771|
|SOWQTQZ12A58A7B63E|Streets On Fire (...|ARPFHN61187FB575F6|   0|279.97995|
|SODUJBS12A8C132150|Wessex Loses a Bride|ARI2JSK1187FB496EF|   0|111.62077|
|SOZHPGD12A8C1394FE|     Baby Come To Me|AR9AWNF1187B9AB0B4|   0|236.93016|
+-----------

In [9]:
songs_table=df.select("song_id","title","artist_id","year","duration")

In [11]:

#output_data = "s3n://projectsparkify/"
#songs_table.write.partitionBy("year","artist_id").parquet("s3a://projectsparkify2/data/songs_table.parquet")
songs_table.write.partitionBy("year","artist_id").parquet("./parquet_area/songs_table")

In [6]:
#song_table_read    = spark.read.parquet("./parquet_area/songs_table")["song_id","title","artist_id","duration"]
#artists_table_read = spark.read.parquet("./parquet_area/artists_table")["artist_id","name"]

In [12]:
artists_table = df.selectExpr("artist_id",
                            "artist_name as name",
                            "artist_location as location",
                            "artist_latitude as latitude","artist_longitude as longitude")

In [13]:
artists_table.write.parquet("./parquet_area/artists_table")

In [14]:
log_data ="./data/log-data/*.json"

In [15]:
df =  spark.read.json(log_data)

In [10]:
df_log_data=  df.toPandas()

In [13]:
df_log_data.to_csv("./log_data.csv")

In [7]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [16]:
df=df.filter(df.page == 'NextSong')

In [17]:
users_table =df.selectExpr("userId as user_id",
                            "firstName as first_name",
                            "lastName as last_name",
                            "gender","level")

In [18]:
users_table.write.parquet("./parquet_area/users_table") 

In [37]:
from pyspark.sql.functions import to_timestamp
#to_timestamp(col("input_timestamp"))
df = df.withColumn("timestamp", to_timestamp(col("ts")/1000.0))


In [38]:
df.select("timestamp","ts").show(2, truncate=False)

+-----------------------+-------------+
|timestamp              |ts           |
+-----------------------+-------------+
|2018-11-15 01:30:26.796|1542241826796|
|2018-11-15 01:41:21.796|1542242481796|
+-----------------------+-------------+
only showing top 2 rows



In [63]:
get_datetime = udf(lambda x: datetime.datetime.fromtimestamp(x))
get_hour = udf(lambda x: x.hour)
df =  df.withColumn("datetime", get_hour(col("timestamp")))

In [64]:
df.select("datetime").show(2, truncate=False)

+--------+
|datetime|
+--------+
|1       |
|1       |
+--------+
only showing top 2 rows



In [65]:
# read log data file
df =  spark.read.json(log_data)

# filter by actions for song plays
df = df.filter(df.page == 'NextSong')

# extract columns for users table    
users_table =df.selectExpr("userId as user_id",
                        "firstName as first_name",
                        "lastName as last_name",
                        "gender","level")

In [66]:
users_table.show(3)

+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     26|      Ryan|    Smith|     M| free|
|     26|      Ryan|    Smith|     M| free|
|     26|      Ryan|    Smith|     M| free|
+-------+----------+---------+------+-----+
only showing top 3 rows



In [108]:
get_timestamp = udf(lambda x: int(x),IntegerType())
df = df.withColumn("start_time", get_timestamp(col("ts")))

In [109]:
df.select("timestamp","ts").show(2)

+---------+-------------+
|timestamp|           ts|
+---------+-------------+
|348567532|1542241826796|
|349222532|1542242481796|
+---------+-------------+
only showing top 2 rows



In [107]:

from pyspark.sql.types import StructType, StructField , DoubleType, StringType, IntegerType, DateType,TimestampType
#get_datetime = udf(lambda x: from_unixtime(x), TimestampType())
#df = df.withColumn('datetime2', from_unixtime('timestamp'))

In [110]:
get_datetime = udf(lambda x: datetime.fromtimestamp(x/1000),DateType())
df =  df.withColumn("datetime", get_datetime(col("ts")))

In [112]:
df.select("start_time","ts","datetime").show(2)

+----------+-------------+----------+
|start_time|           ts|  datetime|
+----------+-------------+----------+
| 348567532|1542241826796|2018-11-15|
| 349222532|1542242481796|2018-11-15|
+----------+-------------+----------+
only showing top 2 rows



In [ ]:
log_data = input_data+"/log-data/*.json"

# read log data file
df =  spark.read.json(log_data)

# filter by actions for song plays
df = df.filter(df.page == 'NextSong')

get_timestamp = udf(lambda x: int(x),IntegerType())
df = df.withColumn("start_time", get_timestamp(col("ts")))
# get timestamp on date time
# df = df.withColumn("timestamp", to_timestamp(col("ts")/1000.0))

# create datetime column from original timestamp column
get_datetime = udf(lambda x: datetime.fromtimestamp(x/1000),DateType())
df =  df.withColumn("datetime", get_datetime(col("ts")))

In [176]:
#time_table.withColumn('hour', functions.dayofweek('datetime'))

#from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, weekday


dict_time_func={"hour":functions.hour,
                "day":functions.dayofmonth,
                "week":functions.weekofyear,
                "month":functions.month,
                "year":functions.year ,
                "weekday":functions.dayofweek}

time_table = df.select("start_time","datetime")

for key,value in dict_time_func.items():
    time_table = time_table.withColumn(key,value("datetime"))
    #time_table.show(2)

In [177]:
time_table.show(2)

+----------+----------+----+---+----+-----+----+-------+
|start_time|  datetime|hour|day|week|month|year|weekday|
+----------+----------+----+---+----+-----+----+-------+
| 348567532|2018-11-15|   0| 15|  46|   11|2018|      5|
| 349222532|2018-11-15|   0| 15|  46|   11|2018|      5|
+----------+----------+----+---+----+-----+----+-------+
only showing top 2 rows



In [138]:
from pyspark.sql import functions

In [171]:
#functions.hour("datetime")
#df.where(col("dt_mvmt").isNotNull())
#get_hour = udf(lambda x: functions.hour(x) )

#time_table.filter(col("datetime").isNotNull()).withColumn("hour_tes",get_hour(col("datetime"))).show(2)

time_table.withColumn('hour', functions.dayofweek('datetime')).show(2)

+----------+----------+----+
|start_time|  datetime|hour|
+----------+----------+----+
| 348567532|2018-11-15|   5|
| 349222532|2018-11-15|   5|
+----------+----------+----+
only showing top 2 rows



In [149]:
time_table.select("datetime").show(2)

+----------+
|  datetime|
+----------+
|2018-11-15|
|2018-11-15|
+----------+
only showing top 2 rows



In [159]:
time_table.filter(col("datetime").isNotNull()).count()

6820